In [1]:
import tensorflow as tf
import numpy
from tensorflow.examples.tutorials.mnist import input_data

#parameters
learning_rate = 0.005
training_epochs = 20
batch_size = 100
SUMMARY_DIR = './mnist'

#load mnist dataset
MNIST = input_data.read_data_sets("./MNIST_data",one_hot=True)

with tf.name_scope('input') as scope:
    X = tf.placeholder(tf.float32, [None, 784], name='image')
    y = tf.placeholder(tf.float32, [None, 10], name = 'label')
    
with tf.variable_scope('layer1') as scope:
    W1 = tf.get_variable("W", shape=[784, 512], initializer=tf.contrib.layers.xavier_initializer())
    b1 = tf.Variable(tf.random_normal([512]))
    L1 = tf.nn.softmax(tf.add(tf.matmul(X,W1),b1))
    
with tf.variable_scope('layer2') as scope:
    W2 = tf.get_variable("W", shape=[512, 20], initializer=tf.contrib.layers.xavier_initializer())
    b2 = tf.Variable(tf.random_normal([20]))
    L2 = tf.nn.softmax(tf.add(tf.matmul(L1,W2),b2))

with tf.variable_scope('layer3') as scope:
    W3 = tf.get_variable("W", shape=[20, 10], initializer=tf.contrib.layers.xavier_initializer())
    b3 = tf.Variable(tf.random_normal([10]))
    y_ = tf.add(tf.matmul(L2, W3), b3)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_, labels = y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
tf.summary.scalar("loss", loss)

#merge all summaries
summary = tf.summary.merge_all()

def get_train_batch(batch_size):
    if not hasattr(get_train_batch, "index"):
        get_train_batch._images = MNIST.train.images
        get_train_batch._labels = MNIST.train.labels
        get_train_batch.size = MNIST.train.images.shape[0]
        get_train_batch.index = 0
        
        random_st = numpy.random.get_state()
        numpy.random.shuffle(get_train_batch._images)
        numpy.random.set_state(random_st)
        numpy.random.shuffle(get_train_batch._labels)
        
    if get_train_batch.index + 128 >= get_train_batch.size:
        train_batch = (get_train_batch._images[get_train_batch.index:get_train_batch.size,],
                       get_train_batch._labels[get_train_batch.index:get_train_batch.size,])
        get_train_batch.index = 0
        
        random_st = numpy.random.get_state()
        numpy.random.shuffle(get_train_batch._images)
        numpy.random.set_state(random_st)
        numpy.random.shuffle(get_train_batch._labels)
    else:
        start = get_train_batch.index
        get_train_batch.index += 128
        end = get_train_batch.index
        train_batch = (get_train_batch._images[start:end], get_train_batch._labels[start:end])
    return train_batch

global_step = 0
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#with tf.Session(config = config) as see:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(SUMMARY_DIR, sess.graph)
    for epoch in range(training_epochs):
        total_batch = int(MNIST.train.num_examples / batch_size)
        avg_loss = 0
        
        for i in range(total_batch):
            #batch_xs, batch_ys = MNIST.train.next_batch(batch_size)
            batch_xs, batch_ys = get_train_batch(batch_size)
            feed_dict = {X: batch_xs, y: batch_ys}
            s, l, _ = sess.run([summary, loss, optimizer], feed_dict=feed_dict)
            writer.add_summary(s, global_step=global_step)
            global_step += 1
            avg_loss += l
        print('Epoch:', '%02d' % (epoch + 1), 'loss =', '{:.6f}'.format(avg_loss / total_batch))
    
    correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    acc = sess.run(accuracy, feed_dict= {X: MNIST.test.images, y: MNIST.test.labels})
    print('Test accuracy:', acc)

Extracting ./MNIST_data\train-images-idx3-ubyte.gz
Extracting ./MNIST_data\train-labels-idx1-ubyte.gz
Extracting ./MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data\t10k-labels-idx1-ubyte.gz
Epoch: 01 loss = 1.329088
Epoch: 02 loss = 0.458111
Epoch: 03 loss = 0.379328
Epoch: 04 loss = 0.322503
Epoch: 05 loss = 0.219824
Epoch: 06 loss = 0.194274
Epoch: 07 loss = 0.173604
Epoch: 08 loss = 0.157446
Epoch: 09 loss = 0.140268
Epoch: 10 loss = 0.131198
Epoch: 11 loss = 0.125750
Epoch: 12 loss = 0.117288
Epoch: 13 loss = 0.107748
Epoch: 14 loss = 0.100604
Epoch: 15 loss = 0.099303
Epoch: 16 loss = 0.091289
Epoch: 17 loss = 0.088796
Epoch: 18 loss = 0.087121
Epoch: 19 loss = 0.080355
Epoch: 20 loss = 0.078453
Test accuracy: 0.9594
